# Ejercicio 2: Top 10 arrival airports in 2013

# Team Science t-ish

## Importamos librerias

In [ ]:
import numpy as np
import pandas as pds
import matplotlib.pyplot as plt
import seaborn as sn
import bz2
import os

In [ ]:
#Empleamos el fichero 'bookings.csv.bz2'

In [ ]:
df_bookings=pd.read_csv('bookings.csv.bz2', sep='^', nrows=10) #Comprobamos las 10 primeras lineas

In [ ]:
df_booking

In [ ]:
df_bookings=pd.read_csv('bookings.csv.bz2', sep='^', memory_map=True, error_bad_lines=False, nrows=10000)

In [ ]:
Total_pasajeros=df_bookings['pax'].sum()
Total_pasajeros

In [ ]:
#Agrupamos por Aeropuertos / Cuantos booking tenemos y el nº de pasajeros, incluidos cancelaciones
Tabla_booking_pasajeros=df_bookings.groupby('arr_port').agg({'arr_port': lambda x: x.value_counts(),\
                                             'pax':'sum'})
Tabla_booking_pasajeros

In [ ]:
#LEEMOS SOLO LAS COLUMNAS arr_port y pax mediante la función rad_csv y el argumento "usecols"
df_bookings = pd.read_csv("bookings.csv.bz2", compression= "bz2", sep = "^",\
                       error_bad_lines = False,\
                       memory_map = True,\
                       usecols = ["arr_port", "pax"])

In [ ]:
df_bookings.head(5)

In [ ]:
df_bookings['pax']=df_bookings['pax'].astype(int)  #Quitamos decimales, pero me encuentro N#A

In [ ]:
df_bookings=df_bookings.fillna({'pax':0})   #Rellenamos con 0 los campos  de pasajeros vacíos

In [ ]:
pasajeros_por_aeropuerto=df_bookings.groupby('arr_port').sum()
pasajeros_por_aeropuerto

In [ ]:
pasajeros_por_aeropuerto.sort_values(by = "pax", ascending= False)[0:10]